In [14]:
# Imports Eurostat

import pandas as pd
import eurostat
from pathlib import Path

In [15]:
# Crear carpeta de salida

outdir = Path("../data/external")
outdir.mkdir(parents=True, exist_ok=True)

In [16]:
# Descargar residuos municipales (kg/hab)

df_env_wasmun = eurostat.get_data_df("env_wasmun", flags=False).reset_index()

df_env_wasmun = df_env_wasmun.rename(columns={"geo\\TIME_PERIOD": "geo"})

df_wasmun_long = df_env_wasmun.melt(
    id_vars=["geo", "wst_oper", "unit"],
    var_name="year",
    value_name="value"
)

df_wasmun_long["year"] = pd.to_numeric(df_wasmun_long["year"], errors="coerce")
df_wasmun_long = df_wasmun_long.dropna(subset=["value"])

df_es_ue = df_wasmun_long[
    (df_wasmun_long["geo"].isin(["ES","EU27_2020"])) &
    (df_wasmun_long["unit"]=="KG_HAB") &
    (df_wasmun_long["wst_oper"]=="GEN")
]

df_es_ue.to_csv(outdir / "eurostat_env_wasmun_ES_UE.csv", index=False)
df_es_ue.head()

,geo,wst_oper,unit,year,value
240,ES,GEN,KG_HAB,NaN,240
241,EU27_2020,GEN,KG_HAB,NaN,241
978,ES,GEN,KG_HAB,NaN,A
979,EU27_2020,GEN,KG_HAB,NaN,A
1716,ES,GEN,KG_HAB,1995.0,505.0


In [17]:
# Descargar reciclaje de envases (%)

# --- Generar CSV con la tasa oficial de reciclaje de envases (RT) para ES y EU27_2020 ---
# Dataset: env_waspacr (indicador de cumplimiento)
# Filtro: waste=W1501 (Total packaging), unit=RT, geo in [ES, EU27_2020]

import pandas as pd
from pathlib import Path
import eurostat

datadir = Path("../data/external")
datadir.mkdir(parents=True, exist_ok=True)
out_csv = datadir / "eurostat_env_waspacr_RT_ES_UE.csv"

# Descarga
df = eurostat.get_data_df(
    "env_waspacr",
    filter_pars={"waste": "W1501", "unit": "RT", "geo": ["ES", "EU27_2020"]},
    flags=False
).reset_index(drop=False)

# Normaliza nombre de geo y pasa a largo si viene en ancho
geo_col = next(c for c in df.columns if c.lower().startswith("geo"))
df = df.rename(columns={geo_col: "geo"})
year_cols = [c for c in df.columns if str(c).isdigit()]

if year_cols:
    tidy = df.melt(
        id_vars=["geo"], value_vars=year_cols,
        var_name="year", value_name="value"
    )
else:
    # por si ya viniera tidy
    tidy = df[["geo", "year", "value"]].copy()

# Tipos y orden
tidy["year"] = pd.to_numeric(tidy["year"], errors="coerce")
tidy["value"] = pd.to_numeric(tidy["value"], errors="coerce")
tidy = (
    tidy.dropna(subset=["year", "value"])
        .sort_values(["geo", "year"])
        .reset_index(drop=True)
)

# Guardar CSV (formato largo: geo,year,value)
tidy.to_csv(out_csv, index=False)
print("✅ CSV guardado en:", out_csv)

# (Opcional) versión “wide” para Excel
wide = tidy.pivot(index="year", columns="geo", values="value").reset_index()
wide_out = datadir / "eurostat_env_waspacr_RT_ES_UE_wide.csv"
wide.to_csv(wide_out, index=False)
print("✅ CSV (wide) guardado en:", wide_out)

# (Opcional) comprobación rápida 2022
print("\nValores 2022 (si existen):")
print(tidy.loc[tidy["year"]==2022].sort_values("geo").to_string(index=False))


✅ CSV guardado en: ..\data\external\eurostat_env_waspacr_RT_ES_UE.csv
✅ CSV (wide) guardado en: ..\data\external\eurostat_env_waspacr_RT_ES_UE_wide.csv

Valores 2022 (si existen):
      geo  year  value
       ES  2022   69.4
EU27_2020  2022   65.3


In [18]:
print("Datos descargados y guardados en:", outdir)

Datos descargados y guardados en: ..\data\external


In [19]:
# --- A1. Compute/load weighted table for 'alides1..6' --- Junta de Andalucia
#  Reasons for discarding food (ranking)
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FuncFormatter
from pathlib import Path

PROC = Path("../data/processed"); PROC.mkdir(parents=True, exist_ok=True)
FIG  = Path("../figures"); FIG.mkdir(parents=True, exist_ok=True)

alides_csv = PROC / "esoc2023_alides_motivos_pct.csv"

if alides_csv.exists():
    alides_pct = pd.read_csv(alides_csv)
else:
    # Necesita df (microdatos ESOC) ya cargado
    assert "df" in globals(), "Load ESOC microdata DataFrame into `df` first."

    # Pesos fep -> float
    w = (df["fep"].astype(str)
                 .str.replace(".", "", regex=False)
                 .str.replace(",", ".", regex=False)
                 .replace({"-9": np.nan, "99": np.nan})
                 .astype(float))
    valid = w.notna() & (w > 0)
    df_w = df.loc[valid].copy()
    w    = w.loc[valid].copy()

    alides_cols = [c for c in df_w.columns if c.startswith("alides")]
    assert alides_cols, "No alides1..alides6 columns found."

    def w_yes(series, weights, yes=1):
        s = pd.to_numeric(series, errors="coerce")
        s = s.where(~s.isin([-9, 9, 99, 999]), np.nan)
        m = s.notna()
        if m.sum() == 0 or weights[m].sum() == 0:
            return np.nan
        return float(((s[m] == yes).astype(int) * weights[m]).sum() / weights[m].sum() * 100)

    res = {c: w_yes(df_w[c], w, 1) for c in alides_cols}
    alides_pct = (pd.Series(res, name="pct")
                    .round(1)
                    .sort_values(ascending=False)
                    .reset_index())
    alides_pct.columns = ["variable", "pct"]

    # Mapear a descripciones si está el diccionario
    if "dicc" in globals() and {"Nombre","Descripción"}.issubset(dicc.columns):
        name_to_desc = dicc.set_index("Nombre")["Descripción"].to_dict()
        alides_pct["label"] = alides_pct["variable"].map(name_to_desc).fillna(alides_pct["variable"])
    else:
        alides_pct["label"] = alides_pct["variable"]

    alides_pct.to_csv(alides_csv, index=False)


In [20]:
# --- B2. Plot: Measures to reduce household waste (ranking) ---
# -- Ensure 'label' exists for medidas and plot nicely --
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FuncFormatter
from pathlib import Path

PROC = Path("../data/processed")
FIG  = Path("../figures"); FIG.mkdir(parents=True, exist_ok=True)

# Cargar si no está en memoria
if "medidas_pct" not in globals():
    medidas_pct = pd.read_csv(PROC / "esoc2023_medidas_pct.csv")

# Crear labels desde diccionario si está disponible; si no, mapeo manual
if "label" not in medidas_pct.columns:
    if "dicc" in globals() and {"Nombre","Descripción"}.issubset(dicc.columns):
        name_to_desc = dicc.set_index("Nombre")["Descripción"].to_dict()
        medidas_pct["label"] = medidas_pct["variable"].map(name_to_desc).fillna(medidas_pct["variable"])
    else:
        manual_labels = {
            "medida1": "Donate/sell items for reuse",
            "medida2": "Use rechargeable batteries",
            "medida3": "Bring own bag/trolley",
            "medida4": "Avoid plastic/excessive packaging",
            "medida5": "Repair appliances/devices",
            "medida6": "Repair clothes/shoes",
            "medida7": "Other",
            "medida8": "None",
        }
        medidas_pct["label"] = medidas_pct["variable"].map(manual_labels).fillna(medidas_pct["variable"])

# (opcional) guardar CSV ya con label
medidas_pct.to_csv(PROC / "esoc2023_medidas_pct.csv", index=False)
